In [1]:
include("./src/REoptLite.jl")
using JuMP
using Xpress
using Main.REoptLite
using JSON


┌ Info: Xpress: Found license file C:\xpressmp\bin\xpauth.xpr
└ @ Xpress C:\Users\jchang2\.julia\packages\Xpress\kcAft\src\license.jl:42
┌ Info: Xpress: Development license detected.
└ @ Xpress C:\Users\jchang2\.julia\packages\Xpress\kcAft\src\license.jl:84


In [2]:
function run(scenario, fname)
    
    m = Model(optimizer_with_attributes(Xpress.Optimizer, "logfile" => "output.log"))
    results = run_reopt(m, REoptInputs(scenario))
    
    open(fname*".json", "w") do f
        JSON.print(f, results)
    end
    
    return results
end

run (generic function with 1 method)

In [3]:
scenario_template = JSON.parsefile("test/scenarios/STM_campus_BAU_sseb_crit.json")

Dict{String,Any} with 7 entries:
  "ElectricUtility" => Dict{String,Any}("outage_durations"=>Any[336],"outage_st…
  "ElectricTariff"  => Dict{String,Any}("urdb_label"=>"5e611fec5457a3665e019406…
  "Site"            => Dict{String,Any}("latitude"=>39.7407,"longitude"=>-105.1…
  "Generator"       => Dict{String,Any}("max_kw"=>0.0,"only_runs_during_grid_ou…
  "ElectricLoad"    => Dict{String,Any}("loads_kw"=>Any[4323.64, 4308.39, 4295.…
  "PV"              => Dict{String,Any}("existing_kw"=>3580.54,"max_kw"=>0.0)
  "Financial"       => Dict{String,Any}("owner_discount_pct"=>0.1,"offtaker_dis…

In [4]:
#tzs = collect(range(17, stop=8760-336, step=336))

In [5]:
#crit_ld = scenario_template["ElectricLoad"]["critical_loads_kw"];

In [6]:
#two_week_ld = Any[]

#for tz in tzs
    #push!(two_week_ld, crit_ld[tz:tz+335])
#end

In [7]:
#maximum(sum(ld) for ld in two_week_ld)

In [8]:
results = Dict[]

for VoLL in range(100, stop=100)
    scenario_dict = copy(scenario_template)
    scenario_dict["ElectricUtility"]["outage_start_timesteps"] = collect(range(17, stop=8760-336, step=336))
    scenario_dict["Financial"]["VoLL"] = VoLL
    s = Scenario(scenario_dict)
    push!(results, run(s, "sseb_BAU"*string(VoLL)))
    println("run $VoLL complete")
end

┌ Info: Checking URDB for 
│   urdb_label = 5e611fec5457a3665e019406
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\urdb.jl:151
┌ Info: Querying PVWatts for prodfactor with 
│   pv.name = PV
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:41
┌ Info: PVWatts success.
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:49
┌ Info: ("p.mg_tech_sizes_equal_grid_sizes", false)
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\constraints\outage_constraints.jl:81
┌ Info: Model built. Optimizing...
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:265
┌ Info: REopt solved with 
│   termination_status(m) = OPTIMAL
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\reopt.jl:274


run 100 complete


In [9]:
r = results

1-element Array{Dict,1}:
 Dict{String,Any}("net_capital_costs" => 0.0,"year_one_demand_cost" => 1.21693653e6,"lcc" => 6.3478475e7,"expected_outage_cost" => 7.335176146885381e6,"year_one_utility_kwh" => 3.61559328e7,"year_one_export_benefit" => 0.0,"year_one_energy_cost" => 1.17965962e6,"PVtoCUR" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],"year_one_energy_produced_PV" => 5.543793e6,"PV_net_fixed_om_costs" => 480241.0…)